In [24]:
!pip3 install apache-flink


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip3 install boto3

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/08/a9/34364af9b27ac3376f948a7ed4bfc1d4422fa22753d7a4b6f85cda549e42/boto3-1.34.107-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.35.0,>=1.34.107 from https://files.pythonhosted.org/packages/89/59/105074130fa22bb0578920cfb25cc4cf4a2348b5ec04f113f7b5575f8528/botocore-1.34.107-py3-none-any.whl.metadata
  Obtaining dependency information for jmespath<2.0.0,>=0.7.1 from https://files.pythonhosted.org/packages/31/b4/b9b800c45527aadd64d5b442f9b932b00648617eb5d63d2c7a6587b7cafc/jmespath-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.11.0,>=0.10.0 from https://files.pythonhosted.org/packages/83/37/395cdb6ee92925fa211e55d8f07b9f93cf93f60d7d4ce5e66fd73f1ea986/s3transfer-0.10.1-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<1.27,>=1.25.4 from https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf1331

In [10]:
import boto3


def create_kinesis_stream(stream_name, shard_count):
    try:
        # Initialize the Kinesis client
        kinesis_client = boto3.client(
            'kinesis', 
            region_name='ap-south-1',
            endpoint_url='http://localhost:4566',
            aws_access_key_id='dummy',  
            aws_secret_access_key='dummy'
        )

        # Create the Kinesis stream
        response = kinesis_client.create_stream(
            StreamName=stream_name,
            ShardCount=shard_count
        )

        # Check for successful response
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            print(f"Kinesis stream '{stream_name}' created with {shard_count} shard(s)")
        else:
            print("Failed to create Kinesis stream")

    except Exception as e:
        print(f"Error: {str(e)}")

def delete_kinesis_stream(stream_name):
    try:
        # Initialize the Kinesis client
        kinesis_client = boto3.client('kinesis')

        # Delete the Kinesis stream
        response = kinesis_client.delete_stream(
            StreamName=stream_name
        )

        # Check for successful response
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            print(f"Kinesis stream '{stream_name}' deleted successfully")
        else:
            print("Failed to delete Kinesis stream")

    except Exception as e:
        print(f"Error: {str(e)}")

In [11]:
create_kinesis_stream("input-stream", 1)

Kinesis stream 'input-stream' created with 1 shard(s)


In [13]:
import datetime
import json
import random
import boto3

STREAM_NAME = "input-stream"


def get_data():
    return {
        'event_time': datetime.datetime.now().isoformat(),
        'ticker': random.choice(['AAPL', 'AMZN', 'MSFT', 'INTC', 'TBV']),
        'price': round(random.random() * 100, 2)
    }


def generate(stream_name, kinesis_client, num_samples):
    for _ in range(num_samples):
        data = get_data()
        print(data)
        kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(data),
            PartitionKey="partitionkey")
if __name__ == '__main__':
    num_samples = 10  # Change this to the desired number of samples
    client = boto3.client(
            'kinesis', 
            region_name='ap-south-1',
            endpoint_url='http://localhost:4566',
            aws_access_key_id='dummy',  
            aws_secret_access_key='dummy'
        )
    
    generate(STREAM_NAME, client, num_samples)

{'event_time': '2024-05-17T15:40:21.397776', 'ticker': 'AAPL', 'price': 44.95}
{'event_time': '2024-05-17T15:40:21.468428', 'ticker': 'AMZN', 'price': 82.2}
{'event_time': '2024-05-17T15:40:21.504897', 'ticker': 'INTC', 'price': 78.13}
{'event_time': '2024-05-17T15:40:21.539296', 'ticker': 'AAPL', 'price': 16.74}
{'event_time': '2024-05-17T15:40:21.660122', 'ticker': 'MSFT', 'price': 20.48}
{'event_time': '2024-05-17T15:40:21.681035', 'ticker': 'AAPL', 'price': 4.53}
{'event_time': '2024-05-17T15:40:21.700507', 'ticker': 'AAPL', 'price': 69.56}
{'event_time': '2024-05-17T15:40:21.718740', 'ticker': 'AMZN', 'price': 92.3}
{'event_time': '2024-05-17T15:40:21.742031', 'ticker': 'AAPL', 'price': 18.17}
{'event_time': '2024-05-17T15:40:21.758645', 'ticker': 'TBV', 'price': 0.83}


In [14]:
import os
os.listdir(".")

['kinesis_part_1',
 '.ipynb_checkpoints',
 'flink-sql-connector-kinesis-1.16.1.jar']

In [34]:
import boto3

# Assuming LocalStack is running on localhost with default port 4566
endpoint_url = 'http://localhost:4566'
region_name = 'ap-south-1'  # This can be any region for LocalStack

# Create a Kinesis client
kinesis_client = boto3.client('kinesis', endpoint_url=endpoint_url, region_name=region_name)

# List all Kinesis streams
response = kinesis_client.list_streams()

In [35]:
# Extract and print stream names
if 'StreamNames' in response:
    print("List of Kinesis streams:")
    for stream_name in response['StreamNames']:
        print(stream_name)
else:
    print("No Kinesis streams found.")

List of Kinesis streams:
input-stream


In [ ]:
# Import necessary libraries
from pyflink.table import EnvironmentSettings, TableEnvironment
import os

# 1. Create a Table Environment
env_settings = EnvironmentSettings.in_streaming_mode()
table_env = TableEnvironment.create(env_settings)

# Define configuration properties for the Flink job
props = [
    {
        "PropertyGroupId": "kinesis.analytics.flink.run.options",
        "PropertyMap": {
            "python": "GettingStarted/getting-started.py",  # Python script to be executed
            "jarfile": "flink-sql-connector-kinesis-1.16.1.jar"  # Flink connector JAR file
        }
    },
    {
        "PropertyGroupId": "consumer.config.0",
        "PropertyMap": {
            "input.stream.name": "input-stream",        # Name of the Kinesis Data Stream
            "flink.stream.initpos": "TRIM_HORIZON",     # Initial position to start reading the stream
            "aws.region": "ap-south-1"                   # AWS region where the stream is located
        }
    }
]

# Function to extract property map from the configuration properties
def property_map(props, property_group_id):
    for prop in props:
        if prop["PropertyGroupId"] == property_group_id:
            return prop["PropertyMap"]

# Function to create a source table definition for Kinesis stream
def create_source_table(table_name, stream_name, region, stream_initpos):
    return f""" 
    CREATE TABLE {table_name} (
        ticker VARCHAR(6),
        price DOUBLE,
        event_time TIMESTAMP(3),
        WATERMARK FOR event_time AS event_time - INTERVAL '5' SECOND
    )
    PARTITIONED BY (ticker)
    WITH (
        'connector' = 'kinesis',
        'stream' = '{stream_name}',
        'aws.region' = '{region}',
        'aws.endpoint' = 'http://localhost:4566',
        'aws.credentials.basic.accesskeyid' = 'dummy',
        'aws.credentials.basic.secretkey' = 'dummy',
        'scan.stream.initpos' = '{stream_initpos}',
        'format' = 'json',
        'json.timestamp-format.standard' = 'ISO-8601'
    ) 
    """

# Main function
def main():
    # Define keys for accessing properties
    input_property_group_key = "consumer.config.0"
    input_stream_key = "input.stream.name"
    input_region_key = "aws.region"
    input_starting_position_key = "flink.stream.initpos"

    # Table name for the source data
    input_table_name = "input_table"

    # Get application properties from the props list
    input_property_map = property_map(props, input_property_group_key)
    input_stream = input_property_map[input_stream_key]
    input_region = input_property_map[input_region_key]
    stream_initpos = input_property_map[input_starting_position_key]

    # Check if running in a local environment and set pipeline.jars configuration
    CURRENT_DIR = os.getcwd()  # Get the current working directory
    table_env.get_config().get_configuration().set_string(
        "pipeline.jars",
        "file:///" + CURRENT_DIR + "/flink-sql-connector-kinesis-1.16.1.jar",
    )

    # 2. Create a source table from a Kinesis Data Stream
    table_env.execute_sql(
        create_source_table(input_table_name, input_stream, input_region, stream_initpos)
    )

    print("Successfully created source tables ")

    # 3. Read data from the source table and print it to the console
    query = f"SELECT * FROM {input_table_name}"
    print(f"Executing query: {query}")
    
    table_result = table_env.execute_sql(query)
    
    # Printing results
    with table_result.collect() as results:
        for result in results:
            print(result)

    if os.environ.get("IS_LOCAL"):
        # In a local environment, execute the job synchronously
        table_env.execute("Getting Started Job")
    else:
        # In a remote environment, execute the job asynchronously and print the job status
        job_client = table_env.execute_async("Getting Started Job")
        print(job_client.get_job_status())


# Entry point for the script
if __name__ == "__main__":
    main()

Successfully created source tables 
Executing query: SELECT * FROM input_table
<Row('AAPL', 44.95, datetime.datetime(2024, 5, 17, 15, 40, 21, 397000))>
<Row('AMZN', 82.2, datetime.datetime(2024, 5, 17, 15, 40, 21, 468000))>
<Row('INTC', 78.13, datetime.datetime(2024, 5, 17, 15, 40, 21, 504000))>
<Row('AAPL', 16.74, datetime.datetime(2024, 5, 17, 15, 40, 21, 539000))>
<Row('MSFT', 20.48, datetime.datetime(2024, 5, 17, 15, 40, 21, 660000))>
<Row('AAPL', 4.53, datetime.datetime(2024, 5, 17, 15, 40, 21, 681000))>
<Row('AAPL', 69.56, datetime.datetime(2024, 5, 17, 15, 40, 21, 700000))>
<Row('AMZN', 92.3, datetime.datetime(2024, 5, 17, 15, 40, 21, 718000))>
<Row('AAPL', 18.17, datetime.datetime(2024, 5, 17, 15, 40, 21, 742000))>
<Row('TBV', 0.83, datetime.datetime(2024, 5, 17, 15, 40, 21, 758000))>
<Row('TBV1', 36.14, datetime.datetime(2024, 5, 17, 16, 33, 39, 8000))>
<Row('INTC1', 98.19, datetime.datetime(2024, 5, 17, 16, 33, 39, 40000))>
<Row('AAPL1', 74.61, datetime.datetime(2024, 5, 17, 

In [29]:
import os

In [30]:
os.environ["AWS_ACCESS_KEY_ID"] = "dummy"
os.environ["AWS_SECRET_ACCESS_KEY"] = "dummy"
os.environ["AWS_REGION"] = "ap-south-1"

In [ ]:
hudi_output_path = 's3a://dXXX/tmp1/'

hudi_sink = f"""
CREATE TABLE customers(
    uuid VARCHAR(36) PRIMARY KEY NOT ENFORCED,
    first_name VARCHAR(50),
    city VARCHAR(50),
    state VARCHAR(50)
)
PARTITIONED BY (`state`)
WITH (
    'connector' = 'hudi',
    'path' = '{hudi_output_path}' ,
    'table.type' = 'MERGE_ON_READ' 
);
"""

# Execute the SQL to create the Hudi table
table_env.execute_sql(hudi_sink)

# Define the data to be inserted into the Hudi table
table_env.execute_sql("""
    INSERT INTO customers
        SELECT * FROM source_table
""").wait()